In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import ndimage
from scipy import signal
import os
import statsmodels.api as sm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import PyPDF2
import io
import cmocean

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
    
## Matplotlib rcparams setup:

matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams['font.size']=11
# matplotlib.rcParams['font.weight']=400

matplotlib.rcParams['image.cmap']='cmo.ice'

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',['011936','FF7D00','225560','BFACAA','D72638','788C9B','A33F00','7CAAB0','685655','EB767C'])
matplotlib.rcParams['axes.formatter.limits']=-3,3
matplotlib.rcParams['axes.labelweight']=300

matplotlib.rcParams['legend.frameon']=False

matplotlib.rcParams['boxplot.whiskers']=(5,95)
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.medianprops.color']='black'

matplotlib.rcParams['errorbar.capsize']=5

matplotlib.rcParams['hist.bins']='auto'

plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
fair_scm = pd.concat([pd.read_csv(x) for x in glob.glob('/home/leachl/Documents/Datasets/RCMIP/rcmip/data/results/phase-1/fair/*default*v1-0-1*')])
fair_scm.columns = [str(x) for x in np.arange(1850,2501)]+list(fair_scm.columns[-6:])
fair_scm = fair_scm.loc[:,list(fair_scm.columns[-6:])+[str(x) for x in np.arange(1850,2501)]]
fair_scm.iloc[:,6:] = fair_scm.iloc[:,6:].apply(pd.to_numeric)
fair_scm.columns = list(fair_scm.columns)[:6] + [int(x) for x in list(fair_scm.columns)[6:]]

/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [3]:
RCMIP_concs = pd.read_csv('/home/leachl/Documents/Datasets/RCMIP/rcmip/data/protocol/rcmip-concentrations-annual-means-v4-0-0.csv')
RCMIP_emms = pd.read_csv('/home/leachl/Documents/Datasets/RCMIP/rcmip/data/protocol/rcmip-emissions-annual-means-v4-0-0.csv')
RCMIP_forc = pd.read_csv('/home/leachl/Documents/Datasets/RCMIP/rcmip/data/protocol/rcmip-radiative-forcing-annual-means-v4-0-0.csv')

In [5]:
fair_scm.Scenario.unique()

array(['esm-ssp534-over', 'ssp460', 'rcp60', 'esm-1pct-brch-2000PgC',
       'esm-rcp26-allGHG', 'esm-ssp460', 'ssp245-stratO3', 'ssp245-aer',
       'hist-sol', 'abrupt-4xCO2', 'ssp126', 'esm-rcp26', 'ssp245',
       'esm-ssp245-allGHG', 'esm-hist-allGHG', 'esm-ssp434', 'esm-rcp45',
       'historical', 'esm-bell-750PgC', 'rcp26', 'esm-ssp534-over-allGHG',
       'hist-CO2', 'hist-aer', 'esm-ssp370-lowNTCF', 'esm-rcp60',
       'ssp434', 'esm-ssp119', 'hist-nat', 'ssp245-GHG', 'ssp119',
       'ssp245-nat', 'esm-pi-CO2pulse', 'esm-ssp370-lowNTCF-aerchemmip',
       '1pctCO2-bgc', 'rcp85', 'esm-ssp585', 'esm-1pct-brch-1000PgC',
       'esm-rcp60-allGHG', 'ssp585', 'esm-rcp85', '1pctCO2-4xext',
       'ssp370-lowNTCF', 'ssp534-over', 'esm-ssp585-allGHG',
       'esm-ssp126-allGHG', 'esm-ssp370', 'esm-hist-cmip5',
       'esm-rcp85-allGHG', 'esm-ssp119-allGHG', 'historical-cmip5',
       'esm-ssp370-lowNTCF-aerchemmip-allGHG', 'esm-piControl-allGHG',
       'esm-ssp460-allGHG', 'esm-ssp2